In [1]:
import os
os.chdir("../")

In [2]:
from src.hf_dataset.dataset import get_banking_77
from src.hf_models.get_model import get_distilbert
from src.config.configuration import ConfigurationManager


c:\Users\asus\miniconda3\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

In [5]:
ds = get_banking_77()

model, tokenizer, config = get_distilbert(task="SequenceClassification",quantization_config=bnb_config,
                                          num_labels=77)

/home/tess/work/deep_learning/transformers/models
False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('"en","osLocale"'), PosixPath('"en","availableLanguages"'), PosixPath('{"userLocale"'), PosixPath('{}}'), PosixPath('"en","defaultMessagesFile"'), PosixPath('"/home/tess/.vscode-server/cli/servers/Stable-91fbdddc47bc9c09064bf7acf133d22631cbf083/server/out/nls.messages.json","locale"'), PosixPath('"en","resolvedLanguage"')}
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
DEBUG: Possible options found for libcudart.so: {PosixPath('/usr/l

/home/tess/miniconda3/envs/genai/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/home/tess/miniconda3/envs/genai/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: /home/tess/miniconda3/envs/genai did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/tess/miniconda3/envs/genai/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!                     If you run into issues with 8-bit matmul, you can try 4-bit quantization: https://huggingface.co/blog/4bit-transformers-bitsandbytes
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [4]:
config_manager = ConfigurationManager()
params = config_manager.get_distilbert_config()


d:\goldy\transformers\models


In [7]:
def foo(**kwargs):
    print(kwargs.items())  
    
foo(hello="goldy", world="hello")

dict_items([('hello', 'goldy'), ('world', 'hello')])


In [5]:

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
ds = ds.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
ds = ds.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

ds["train"] = ds["train"].remove_columns(["text"])
ds["test"] = ds["test"].remove_columns(["text"])

In [6]:
from peft import LoraConfig


lora_config = LoraConfig(
    r = 7, # low rank means smaller update matrices with fewer trainable parameters
    target_modules = ["q_lin", "k_lin"],
    bias="none",
    lora_alpha=32,
    
)


In [ ]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [8]:
model

PeftModel(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=7, bias=False)
             

In [9]:
model.print_trainable_parameters()

trainable params: 129,024 || all params: 67,141,709 || trainable%: 0.1922


In [16]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import EvalPrediction
import numpy as np

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    torch_compile=True
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    compute_metrics=compute_accuracy,
)

In [17]:
ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3080
    })
})

In [18]:
trainer.train()

W0306 23:08:14.695821 14992 site-packages\torch\_inductor\utils.py:1137] [6/0] Not enough SMs to use max_autotune_gemm mode


BackendCompilerFailed: backend='inductor' raised:
RuntimeError: Cannot find a working triton installation. Either the package is not installed or it is too old. More information on installing Triton can be found at https://github.com/openai/triton

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [15]:
trainer.evaluate(ds["test"])

{'eval_runtime': 6.9186,
 'eval_samples_per_second': 445.178,
 'eval_steps_per_second': 14.02,
 'epoch': 4.0}

In [19]:
model

PeftModel(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=7, bias=False)
             